In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("drive/MyDrive/Computer Vision/sdxl")
os.chdir("diffusers/src")
!ls

Mounted at /content/drive/
Demo_Particle_Guidance	generated_images	      ref_images
diffusers		particle_guidance_demo.ipynb  Untitled0.ipynb


##Use a graphics card with at least 30 GPU RAM. If you're using Google Colab, select A100.

# Vanilla I.I.D Sampling

In [ ]:
# make sure you're logged in with \`huggingface-cli login\`
from diffusers.pipelines.stable_diffusion_xl.pipeline_stable_diffusion_xl import StableDiffusionXLPipeline
from diffusers import LMSDiscreteScheduler, DDIMScheduler, DDPMScheduler, EulerDiscreteScheduler
import torch
import argparse
import os
from torchvision.utils import make_grid, save_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

prompt_lists = [
    ["Bustling city street"]
]

from torchvision.utils import save_image
import os


seed = 6
num_of_image = 4
# number of sampling steps
steps = 30
# guidance scale
w = 4
output_dir = "generated_images/"  # Replace with your directory


generator = torch.Generator(device="cuda").manual_seed(seed)

pipe = StableDiffusionXLPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float32
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


pipe = pipe.to(device)

pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

for j, prompt in enumerate(prompt_lists):
  prompt *= 4

  out = pipe(prompt, generator=generator, num_inference_steps=steps, guidance_scale=w, output_type='tensor')

  image = out.images[0]
  #for i, img in enumerate(image):
   #   img_torch = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0)  # Convert to PyTorch format
    #  save_image(img_torch, os.path.join(output_dir, f'sdxl_normal_{j, i}.png'))  # Save the image
  image_grid = make_grid(torch.from_numpy(image).permute(0, 3, 1, 2), nrow=int(np.sqrt(len(image))))
  plt.imshow(image_grid.permute(1, 2, 0))

del out
torch.cuda.empty_cache()

# Particle Guidance in Stable Diffusion XL

In [ ]:
from diffusers.pipelines.stable_diffusion_xl.pipeline_stable_diffusion_xl_diffusion_particle import StableDiffusionXLParticlePipeline

# make sure you're logged in with \`huggingface-cli login\`
from diffusers import LMSDiscreteScheduler, DDIMScheduler, DDPMScheduler, EulerDiscreteScheduler
import torch
import argparse
import os
from torchvision.utils import make_grid, save_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

prompt_lists = [
    ["VAN GOGH CAFE TERASSE"]
]

from torchvision.utils import save_image
import os


seed = 3
num_of_image = 4
# number of sampling steps
steps = 30
# guidance scale
w = 4
output_dir = "generated_images/"  # Replace with your directory
# coeff for particle guidance
coeff=70

generator = torch.Generator(device="cuda").manual_seed(seed)

pipe = StableDiffusionXLParticlePipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float32
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


pipe = pipe.to(device)

pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

for j, prompt in enumerate(prompt_lists):
  prompt *= 4

  out = pipe(prompt, generator=generator, num_inference_steps=steps, coeff= coeff, guidance_scale=w, output_type='tensor')

  image = out.images[0]
  #for i, img in enumerate(image):
   #   img_torch = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0)  # Convert to PyTorch format
    #  save_image(img_torch, os.path.join(output_dir, f'sdxl_coeff_{coeff}_{j, i}.png'))  # Save the image
  image_grid = make_grid(torch.from_numpy(image).permute(0, 3, 1, 2), nrow=int(np.sqrt(len(image))))
  plt.imshow(image_grid.permute(1, 2, 0))

del out
torch.cuda.empty_cache()

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]